<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-знакомство-с-данными" data-toc-modified-id="Загрузка-и-знакомство-с-данными-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и знакомство с данными</a></span></li><li><span><a href="#EDA" data-toc-modified-id="EDA-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>EDA</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Исследование задачи</a></span><ul class="toc-item"><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li></ul></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span><ul class="toc-item"><li><span><a href="#Взвешивание-классов" data-toc-modified-id="Взвешивание-классов-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Взвешивание классов</a></span><ul class="toc-item"><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-5.1.1"><span class="toc-item-num">5.1.1&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-5.1.2"><span class="toc-item-num">5.1.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-5.1.3"><span class="toc-item-num">5.1.3&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li></ul></li><li><span><a href="#Увеличение-выборки" data-toc-modified-id="Увеличение-выборки-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Увеличение выборки</a></span><ul class="toc-item"><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-5.2.1"><span class="toc-item-num">5.2.1&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-5.2.2"><span class="toc-item-num">5.2.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-5.2.3"><span class="toc-item-num">5.2.3&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li></ul></li><li><span><a href="#Увеличение-порога" data-toc-modified-id="Увеличение-порога-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Увеличение порога</a></span><ul class="toc-item"><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-5.3.1"><span class="toc-item-num">5.3.1&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-5.3.2"><span class="toc-item-num">5.3.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-5.3.3"><span class="toc-item-num">5.3.3&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li></ul></li></ul></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Загрузка и знакомство с данными

Перед началолом работы разрузим все необходимые библиотеки.

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, confusion_matrix, plot_confusion_matrix, recall_score, mean_absolute_error
from sklearn.metrics import roc_curve, roc_auc_score, r2_score, precision_score, f1_score, precision_recall_curve
from sklearn.utils import shuffle

import matplotlib.pyplot as plt
from IPython.display import display
import seaborn as sns
import numpy as np
from sklearn.metrics import accuracy_score
import warnings
import re
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

warnings.simplefilter(action='ignore', category=FutureWarning)

Загрузим файл в датафрейм. Так как я работаю локально, потренировалась в загрузке датасета напрямую из kaggle:

Посмотрим размерность датасета и выведем несколько строк для наглядности данных. Немного изменим написание названия столбцов для бОльшей читаемости.

In [2]:
# df = pd.read_csv('/datasets/Churn.csv')
df = pd.read_csv('C:/Users/valentina.tikhova/Downloads/yandex_ds/course7_Обучение с учителем/Churn.csv')
df.columns = [re.sub(r"\B([A-Z])", r"_\1", col).lower() for col in df.columns]

display(df.shape)
display(df.sample(3))

(10000, 14)

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
4206,4207,15720929,Kazantseva,604,France,Female,47,8.0,62094.71,3,0,0,9308.10,1
5902,5903,15586504,Trevisani,694,France,Male,40,9.0,0.00,2,1,0,40463.03,0
5930,5931,15645744,Chukwudi,826,France,Female,30,5.0,0.00,2,0,1,157397.57,0


В нашем распоряжении 10k записей, каждый объект в наборе данных — информация о клиента банка, а именно:
- `row_number` — индекс строки в данных
- `customer_id` — уникальный идентификатор клиента
- `surname` — фамилия
- `credit_score` — кредитный рейтинг
- `geography` — страна проживания
- `gender` — пол
- `age` — возраст
- `tenure` — сколько лет человек является клиентом банка
- `balance` — баланс на счёте
- `num_of_products` — количество продуктов банка, используемых клиентом
- `has_cr_card` — наличие кредитной карты
- `is_active_member` — активность клиента
- `estimated_salary` — предполагаемая зарплата
- `exited` — факт ухода клиента


## EDA


Посмотрим информацию по выборке методом describe(), чтобы воспринимать информацию лучше, покажем отдельно информацию о категориальных и количественных переменных. На локальной машине можно запустить `pandas_profiling` -- полная информация о датафрейме.

Из категориальных признаков у нас фамилия и пол клиента, а так же страна проживания. 

In [3]:
df.describe(include = np.object)

,surname,geography,gender
count,10000,10000,10000
unique,2932,3,2
top,Smith,France,Male
freq,32,5014,5457


В данных нет пропусков, представлены три страны проживания клиентов, лидирует из них Франция (порядка 5 тыс с небольшим респондентов живут в данной стране). В выборке больше всего мужчин, а самая распространненая фамилия клиентов - Смит.

Посмотрим на количественные признаки в дата-сете.

In [4]:
df.describe(include = np.number).round(2)

,row_number,customer_id,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
count,10000.00,10000.00,10000.00,10000.00,9091.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.0
mean,5000.50,15690940.57,650.53,38.92,5.00,76485.89,1.53,0.71,0.52,100090.24,0.2
std,2886.90,71936.19,96.65,10.49,2.89,62397.41,0.58,0.46,0.50,57510.49,0.4
min,1.00,15565701.00,350.00,18.00,0.00,0.00,1.00,0.00,0.00,11.58,0.0
25%,2500.75,15628528.25,584.00,32.00,2.00,0.00,1.00,0.00,0.00,51002.11,0.0
50%,5000.50,15690738.00,652.00,37.00,5.00,97198.54,1.00,1.00,1.00,100193.92,0.0
75%,7500.25,15753233.75,718.00,44.00,7.00,127644.24,2.00,1.00,1.00,149388.25,0.0
max,10000.00,15815690.00,850.00,92.00,10.00,250898.09,4.00,1.00,1.00,199992.48,1.0


По выведенной информации видим пропуски в столбце `tenure`, заполним его медианным значением методом `fillna`.

In [5]:
df = df.fillna(df.median())

Поля с индексами строки и фамилией не являются важной информацией для предсказания оттока клиента из банка -- ими можно пренебречь. К тому же уникальным идентификатором клиента может быть индекс строки.

In [6]:
df = df.drop(['row_number', 'surname', 'customer_id'] , axis=1)

Проверим наличие дубликатов в выборке:

In [7]:
df.duplicated().sum()

0

Поменяем тип данных в столбцах на основе значений объекта в каждом столбце с помощью метода `convert_dtypes()`.

In [8]:
df = df.convert_dtypes()
df.dtypes

credit_score          Int64
geography            string
gender               string
age                   Int64
tenure                Int64
balance             Float64
num_of_products       Int64
has_cr_card           Int64
is_active_member      Int64
estimated_salary    Float64
exited                Int64
dtype: object

Как видим, страна и пол клиента теперь строковые переменные. Заменим вещественный тип данных на целочисленный.

In [9]:
for i in ['tenure', 'balance', 'estimated_salary']:
    df[i] = df[i].astype('int')

Посмотрим, какие параметры влиют больше всего на факт оттока из банка с помощью функции `corr()`. 

In [10]:
df.corr().style.background_gradient(cmap='coolwarm', 
                                    axis=None, 
                                    vmin=df.corr().min().min(), 
                                    vmax=df.corr().max().max()).set_precision(3)

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
credit_score,1.000,-0.004,-0.000,0.006,0.012,-0.005,0.026,-0.001,-0.027
age,-0.004,1.000,-0.013,0.028,-0.031,-0.012,0.085,-0.007,0.285
tenure,-0.000,-0.013,1.000,-0.008,0.011,0.026,-0.031,0.010,-0.016
balance,0.006,0.028,-0.008,1.000,-0.304,-0.015,-0.010,0.013,0.119
num_of_products,0.012,-0.031,0.011,-0.304,1.000,0.003,0.010,0.014,-0.048
has_cr_card,-0.005,-0.012,0.026,-0.015,0.003,1.000,-0.012,-0.010,-0.007
is_active_member,0.026,0.085,-0.031,-0.010,0.010,-0.012,1.000,-0.011,-0.156
estimated_salary,-0.001,-0.007,0.010,0.013,0.014,-0.010,-0.011,1.000,0.012
exited,-0.027,0.285,-0.016,0.119,-0.048,-0.007,-0.156,0.012,1.000


На факт оттока клиента больше всего влияет его возраст, а также баланс счета. Интересно, что чем активнее клиент, тем выше вероятность его ухода. Можно объяснить это заинтересованностью клиента в выгодных условиях банковских услуг. Вероятно, чем больше клиент использует банковские услуги, тем больше сравнивает их уровень у конкурентов.

Все же сильных взаимосвязей между признаками нет, ничего не будет менять будущей модели.

Чтобы составить начальный портрет клиентов, ушедших в отток, и лояльных клиентов, которые продолжаются пользоваться услугами банка, посмотрим всю ту же информацию `describe()` по целевому признаку `exited`:

In [11]:
df.query("exited == 1").describe(percentiles=[.50, .9], include = 'all').round(2).fillna('-')

,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
count,2037.0,2037,2037,2037.0,2037.0,2037.0,2037.0,2037.0,2037.0,2037.0,2037.0
unique,-,3,2,-,-,-,-,-,-,-,-
top,-,Germany,Female,-,-,-,-,-,-,-,-
freq,-,814,1139,-,-,-,-,-,-,-,-
mean,645.35,-,-,44.84,4.91,91108.17,1.48,0.7,0.36,101465.18,1.0
std,100.32,-,-,9.76,2.81,58360.6,0.8,0.46,0.48,57912.42,0.0
min,350.0,-,-,18.0,0.0,0.0,1.0,0.0,0.0,11.0,1.0
50%,646.0,-,-,45.0,5.0,109349.0,1.0,1.0,0.0,102460.0,1.0
90%,776.4,-,-,58.0,9.0,152080.0,3.0,1.0,1.0,180168.8,1.0
max,850.0,-,-,84.0,10.0,250898.0,4.0,1.0,1.0,199808.0,1.0


**Портрет пользователей, ушедших в отток**:
- большинство (90%) имеют кредитный скоринг более 770
- в оттоке в основном - женщины за 58 лет, проживающие в Германии
- в среднем были клиентами банка в течение 4.8 лет
- средний баланс на счету клиента - 91 тыс у.е., предполагаемая зп - 101 тыс у.е.
- большинство имеют около трех банковских продуктов

In [12]:
df.query("exited == 0").describe(percentiles=[.50, .9], include = 'all').round(2).fillna('-')

,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
count,7963.0,7963,7963,7963.0,7963.0,7963.0,7963.0,7963.0,7963.0,7963.0,7963.0
unique,-,3,2,-,-,-,-,-,-,-,-
top,-,France,Male,-,-,-,-,-,-,-,-
freq,-,4204,4559,-,-,-,-,-,-,-,-
mean,651.85,-,-,37.41,5.02,72744.99,1.54,0.71,0.55,99737.9,0.0
std,95.65,-,-,10.13,2.75,62847.82,0.51,0.46,0.5,57405.58,0.0
min,405.0,-,-,18.0,0.0,0.0,1.0,0.0,0.0,90.0,0.0
50%,653.0,-,-,36.0,5.0,92072.0,2.0,1.0,1.0,99645.0,0.0
90%,778.0,-,-,49.0,9.0,148729.4,2.0,1.0,1.0,179452.6,0.0
max,850.0,-,-,92.0,10.0,221532.0,3.0,1.0,1.0,199992.0,0.0


**Портрет оставшихся пользователей**:
- в большинстве случаев - французы до 50 лет с кредитным скорингом -- порядка 778
- в среднем являются клиентами банка около 5 лет
- средний баланс на счету клиента - 73 тыс у.е., предполагаемая зп - 100 тыс у.е.
- большинство имеют около двух банковских продуктов

Оба типа клиента (по факту оттока) почти не отличаются количеством лет `tenure`, а также фактом наличия кредитной карты. Это также подтвеждается коррелацией признаков.

## Подготовка данных

Перед созданием модели необходимо проверсти подготовку данных:
- кодирование категориальных переменных;
- разбиение на выборки;
- нормирование числовых данных.
    
Как увидели выше, в датасете есть категориальные признаки (география клиента и его пол), которые необходимо преобразовать в численные. Например, методом прямого кодирования `OneHotEncoding`.

In [13]:
data_ohe = pd.get_dummies(df, columns=['geography', 'gender'], drop_first=True)
data_ohe.sample(5)

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited,geography_Germany,geography_Spain,gender_Male
5148,731,70,3,0,2,1,1,141180,0,0,1,1
5941,612,33,0,64900,2,1,0,102426,0,0,0,0
1302,428,31,5,0,2,1,0,54487,0,0,0,0
913,756,27,1,131899,1,1,0,93302,0,1,0,1
7872,639,46,10,110031,2,1,1,133995,0,1,0,0


In [14]:
for column in data_ohe.columns:
    data_ohe[column] = data_ohe[column].astype('int8')

Зафиксируем псевдослучайность для всех используемых в проекте алгоритмов:

In [15]:
rnd_st = 12345

Напишем функции для дальнейшего использования:
* **target_features** : выделим целевой признак `target` и признаки для обучения модели `features`
* **split_data** : разобьем выборку с помощью метода `train_test_split` в пропорции 60\20\20 на 
    - обучающую *_train
    - валидационную *_valid
    - тестовую выборки *_test

 и выполним масштабирование количественных признаков (скаллер обучаем на обучающей выборке и применяем ко всем)

In [16]:
def target_features(data_ohe, target_column='exited'):
    '''Функция возвращает целевой признак и признаки для обучения модели'''    
    target = data_ohe[target_column]
    features = data_ohe.drop(target_column, axis=1)
    return target, features

In [17]:
def split_data(features, target):
    '''Функция разделения выборки'''
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, train_size=0.6, test_size=0.4, random_state=rnd_st)

    features_test, features_valid, target_test, target_valid = train_test_split(
        features_valid, target_valid, test_size=0.5, random_state=rnd_st)
    
    scaler = StandardScaler()
    numeric = ['credit_score', 'age', 'balance', 'estimated_salary', 'tenure', 'num_of_products']
    scaler.fit(features_train[numeric])
    
    features_train[numeric] = scaler.transform(features_train[numeric])
    features_valid[numeric] = scaler.transform(features_valid[numeric])
    features_test[numeric] = scaler.transform(features_test[numeric])
    
    return features_train, features_valid, features_test, target_train, target_valid, target_test

Данные готовы к исследованию.

## Исследование задачи

Посмотрим, в каком соотношении встречается целевой признак.

In [18]:
data_ohe['exited'].value_counts(normalize=True)

0    0.7963
1    0.2037
Name: exited, dtype: float64

Видим, что соотношение классов далеко от 1:1, т.е. они несбалансированы: ~80% отрицательных и ~20% положительных.

При бисбалансе классов применим несколько моделий обучения и посмотрим, какая из них покажет лучшие результаты метрик f1 и roc_auc. Для этого напишем функцию `check_model`.

In [19]:
def check_model(model, test = None, samplefunc = None, threshold = None):
    '''Функция возвращает модель и ее показатели:
       - выделяет целевой признак и признаки модели
       - делит выборку на обучающую, валидационную, тестовую
       - применяет к выборкам функцию увеличения, если необходимо
       - использует валидационную выборку, если не указана тестовая
       - выбирает лучший порог
       - показывает метрики f1 и auc модели 
    '''
    
    target, features = target_features(data_ohe)
    
    features_train, features_valid, features_test, target_train, target_valid, target_test = split_data(features, target)
    
    if samplefunc:
        features_train, target_train = samplefunc(features_train, target_train)
        
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    roc_auc = roc_auc_score(target_valid, predicted_valid, multi_class='ovr')
    
    if test:
        predicted_test = model.predict(features_test)
        f1 = f1_score(target_test, predicted_test)
        roc_auc = roc_auc_score(target_test, predicted_test, multi_class='ovr')
        
    if threshold:
        probabilities_valid = model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        best_th = 0
        best_f1 = 0
        for threshold in np.arange(0, 0.7, 0.05):
            predicted_valid = probabilities_one_valid > threshold
            predicted = model.predict(features) 
            f1 = f1_score(target_valid, predicted_valid)
            roc_auc = roc_auc_score(target_valid, predicted_valid, multi_class='ovr')
            if f1 > best_f1:
                best_f1 = f1
                best_th = threshold
        print(' | Лучший порог= {:.2f}'.format(threshold), end='')        
            
    print(" | f1 = {:.5f} | roc_auc = {:.5f}".format(f1, roc_auc))

### Дерево решений

In [20]:
for max_depth in range(2, 22, 2):
    print("max_depth = ", max_depth, end='')
    model = check_model(DecisionTreeClassifier(max_depth=max_depth, random_state=rnd_st))    

max_depth =  2 | f1 = 0.49863 | roc_auc = 0.67550
max_depth =  4 | f1 = 0.48074 | roc_auc = 0.66220
max_depth =  6 | f1 = 0.52067 | roc_auc = 0.68192
max_depth =  8 | f1 = 0.47493 | roc_auc = 0.66050
max_depth =  10 | f1 = 0.48468 | roc_auc = 0.66731
max_depth =  12 | f1 = 0.47471 | roc_auc = 0.66400
max_depth =  14 | f1 = 0.47415 | roc_auc = 0.66452
max_depth =  16 | f1 = 0.47202 | roc_auc = 0.66432
max_depth =  18 | f1 = 0.48077 | roc_auc = 0.67014
max_depth =  20 | f1 = 0.48980 | roc_auc = 0.67582


Пока лучшие результаты f1=0.52067 получили у модели с глубиной равной 6.

### Случайный лес

В качестве гиперпараметра глубины дерева возьмем значение, найденное для предыдущей модели. 
А количество деревьев для нашего случайного леса будет искать в диапазоне от 10 до 100 с шагом 10.

In [21]:
for estim in range(10, 101, 10):
    print("estim = ", estim, end='')
    model = check_model(RandomForestClassifier(n_estimators=estim, max_depth=6, random_state=rnd_st))

estim =  10 | f1 = 0.45409 | roc_auc = 0.64807
estim =  20 | f1 = 0.45118 | roc_auc = 0.64666
estim =  30 | f1 = 0.45638 | roc_auc = 0.64903
estim =  40 | f1 = 0.46589 | roc_auc = 0.65344
estim =  50 | f1 = 0.46465 | roc_auc = 0.65266
estim =  60 | f1 = 0.47315 | roc_auc = 0.65652
estim =  70 | f1 = 0.46599 | roc_auc = 0.65306
estim =  80 | f1 = 0.45095 | roc_auc = 0.64629
estim =  90 | f1 = 0.45283 | roc_auc = 0.64715
estim =  100 | f1 = 0.45392 | roc_auc = 0.64770


Как видим, модель случайного леса предсказывает не лучше, чем дерево решений. Лучший результат 0.47315 достигается при количестве деревьев равном 60.

### Логистическая регрессия

In [22]:
check_model(LogisticRegression(random_state = rnd_st))

 | f1 = 0.27225 | roc_auc = 0.56937


При дисбалансе классов логистическая регрессия предсказывает крайне плохо.

Самая лучшая модель -- дерево решений, с учетом дисбаланс классов должна показать хорошие результаты.

## Борьба с дисбалансом

### Взвешивание классов

Придадим объектам редкого класса 1 больший вес используя гиперпараметр class_weight='balanced'.

#### Дерево решений

In [23]:
for max_depth in range(2, 22, 2):
    print("max_depth = ", max_depth, end='')
    model = check_model(DecisionTreeClassifier(class_weight='balanced', max_depth=max_depth, random_state=rnd_st))  

max_depth =  2 | f1 = 0.51297 | roc_auc = 0.70432
max_depth =  4 | f1 = 0.53528 | roc_auc = 0.73788
max_depth =  6 | f1 = 0.57116 | roc_auc = 0.75272
max_depth =  8 | f1 = 0.57576 | roc_auc = 0.75503
max_depth =  10 | f1 = 0.54416 | roc_auc = 0.72357
max_depth =  12 | f1 = 0.51564 | roc_auc = 0.69849
max_depth =  14 | f1 = 0.46400 | roc_auc = 0.66101
max_depth =  16 | f1 = 0.45858 | roc_auc = 0.65633
max_depth =  18 | f1 = 0.45006 | roc_auc = 0.65097
max_depth =  20 | f1 = 0.45545 | roc_auc = 0.65377


Лучшая модель с глубиной дерева -- 8 (f1 = 0.57576).

#### Случайный лес

In [24]:
for est in range(10, 101, 10):
    print("est = ", est, end='')
    model = check_model(RandomForestClassifier(class_weight='balanced', 
                                               n_estimators=est, 
                                               max_depth=8, 
                                               random_state=rnd_st))

est =  10 | f1 = 0.58155 | roc_auc = 0.74487
est =  20 | f1 = 0.57855 | roc_auc = 0.74173
est =  30 | f1 = 0.58509 | roc_auc = 0.74334
est =  40 | f1 = 0.58927 | roc_auc = 0.74790
est =  50 | f1 = 0.59006 | roc_auc = 0.74744
est =  60 | f1 = 0.58824 | roc_auc = 0.74571
est =  70 | f1 = 0.58875 | roc_auc = 0.74680
est =  80 | f1 = 0.58875 | roc_auc = 0.74680
est =  90 | f1 = 0.59071 | roc_auc = 0.74775
est =  100 | f1 = 0.59251 | roc_auc = 0.74948


А вот модель случайного леса показывает самое высокое значение F1-меры: 0.59251 при количестве деревьев равном 100.

#### Логистическая регрессия

In [25]:
check_model(LogisticRegression(class_weight='balanced', random_state = rnd_st))

 | f1 = 0.48751 | roc_auc = 0.69025


Особых улучшений нет.

### Увеличение выборки
Cделаем объекты редкого класса не такими редкими и переобучим модели.

Попробуем увеличить `upsample` выборку для увеличения метрик. Для этого напишем соответствующую функцию и попробуем подобрать лучшее значение увеличения выборки с помощью функции `map`.

In [26]:
def upsampled(features, target, repeat=10):
    ''' Функция увеличения выборки для увеличения метрик качества'''    
    features_0 = features[target == 0]
    features_1 = features[target == 1]
    target_0 = target[target == 0]
    target_1 = target[target == 1]
    
    # увеличение
    features_up = pd.concat([features_0] + [features_1] * repeat)
    target_up = pd.concat([target_0] + [target_1] * repeat)
    
    # перемешиваем
    features_up, target_up = shuffle(features_up, target_up, random_state=rnd_st)  
    
    return features_up, target_up

#### Дерево решений

In [27]:
for max_depth in range(2, 22, 2):
    print("max_depth = ", max_depth, end='')
    model = check_model(DecisionTreeClassifier(max_depth=max_depth, random_state=rnd_st), 
                        samplefunc = upsampled) 

max_depth =  2 | f1 = 0.48220 | roc_auc = 0.69393
max_depth =  4 | f1 = 0.43615 | roc_auc = 0.65233
max_depth =  6 | f1 = 0.49031 | roc_auc = 0.70965
max_depth =  8 | f1 = 0.48842 | roc_auc = 0.70399
max_depth =  10 | f1 = 0.50079 | roc_auc = 0.70816
max_depth =  12 | f1 = 0.51459 | roc_auc = 0.71176
max_depth =  14 | f1 = 0.51085 | roc_auc = 0.70088
max_depth =  16 | f1 = 0.49498 | roc_auc = 0.68251
max_depth =  18 | f1 = 0.47740 | roc_auc = 0.66931
max_depth =  20 | f1 = 0.46494 | roc_auc = 0.65968


Самый лучший результат у модели с 12 деревьями (f1 = 0.51459).

#### Случайный лес

In [28]:
for est in range(10, 101, 10):
    print("est = ", est, end='')
    model = check_model(RandomForestClassifier(n_estimators=est, 
                                               max_depth=12, 
                                               random_state=rnd_st), samplefunc = upsampled)

est =  10 | f1 = 0.56324 | roc_auc = 0.74049
est =  20 | f1 = 0.57339 | roc_auc = 0.74932
est =  30 | f1 = 0.57782 | roc_auc = 0.75341
est =  40 | f1 = 0.58870 | roc_auc = 0.75943
est =  50 | f1 = 0.58766 | roc_auc = 0.76013
est =  60 | f1 = 0.58812 | roc_auc = 0.75912
est =  70 | f1 = 0.58812 | roc_auc = 0.75912
est =  80 | f1 = 0.58225 | roc_auc = 0.75384
est =  90 | f1 = 0.58276 | roc_auc = 0.75494
est =  100 | f1 = 0.58391 | roc_auc = 0.75557


Самый лучший результат у модели с глубиной 40 -- f1 = 0.58870.

#### Логистическая регрессия

In [29]:
check_model(LogisticRegression(random_state = rnd_st), samplefunc = upsampled)

 | f1 = 0.42082 | roc_auc = 0.63116


### Увеличение порога

Посмотрим лучший порог на каждой модели

#### Дерево решений

In [30]:
for max_depth in range(2, 22, 2):
    print("max_depth = ", max_depth, end='')
    model = check_model(DecisionTreeClassifier(max_depth=max_depth, random_state=rnd_st), 
                        threshold = 1) 

max_depth =  2 | Лучший порог= 0.65 | f1 = 0.09692 | roc_auc = 0.52315
max_depth =  4 | Лучший порог= 0.65 | f1 = 0.37956 | roc_auc = 0.61627
max_depth =  6 | Лучший порог= 0.65 | f1 = 0.44750 | roc_auc = 0.64479
max_depth =  8 | Лучший порог= 0.65 | f1 = 0.45342 | roc_auc = 0.64880
max_depth =  10 | Лучший порог= 0.65 | f1 = 0.46418 | roc_auc = 0.65566
max_depth =  12 | Лучший порог= 0.65 | f1 = 0.45614 | roc_auc = 0.65252
max_depth =  14 | Лучший порог= 0.65 | f1 = 0.47328 | roc_auc = 0.66374
max_depth =  16 | Лучший порог= 0.65 | f1 = 0.47202 | roc_auc = 0.66432
max_depth =  18 | Лучший порог= 0.65 | f1 = 0.48077 | roc_auc = 0.67014
max_depth =  20 | Лучший порог= 0.65 | f1 = 0.48980 | roc_auc = 0.67582


#### Случайный лес

In [31]:
for est in range(10, 101, 10):
    print("est = ", est, end='')
    model = check_model(RandomForestClassifier(n_estimators=est, 
                                               max_depth=12, 
                                               random_state=rnd_st), threshold = 1)

est =  10 | Лучший порог= 0.65 | f1 = 0.39572 | roc_auc = 0.62265
est =  20 | Лучший порог= 0.65 | f1 = 0.40285 | roc_auc = 0.62564
est =  30 | Лучший порог= 0.65 | f1 = 0.40285 | roc_auc = 0.62564
est =  40 | Лучший порог= 0.65 | f1 = 0.41071 | roc_auc = 0.62896
est =  50 | Лучший порог= 0.65 | f1 = 0.42908 | roc_auc = 0.63668
est =  60 | Лучший порог= 0.65 | f1 = 0.42553 | roc_auc = 0.63519
est =  70 | Лучший порог= 0.65 | f1 = 0.43110 | roc_auc = 0.63755
est =  80 | Лучший порог= 0.65 | f1 = 0.42756 | roc_auc = 0.63605
est =  90 | Лучший порог= 0.65 | f1 = 0.41577 | roc_auc = 0.63109
est =  100 | Лучший порог= 0.65 | f1 = 0.42199 | roc_auc = 0.63369


#### Логистическая регрессия

In [32]:
check_model(LogisticRegression(random_state = rnd_st), threshold = 1)

 | Лучший порог= 0.65 | f1 = 0.11839 | roc_auc = 0.52612


Лучший уровень порога для лубой модели -- 0.65, при этом метрика f1 не дает результатов лучше, яем другие методы борьбы с дисбалансом.

**Как итог, лучший примененный метод -- взвешивание классов, а лучшая модель -- случайный лес с глубиной 100.**

## Тестирование модели

Проверим самую лучшую модель по итогам прошлого пункта (`случайный лес`) с количеством деревьев равным 1000 и глубиной равной 8 с учетом балансировки классов.

In [33]:
check_model(RandomForestClassifier(class_weight='balanced', n_estimators=100, max_depth=8, random_state=rnd_st))

 | f1 = 0.59251 | roc_auc = 0.74948


Протестируем модель на тестовой выборке:

In [34]:
check_model(RandomForestClassifier(class_weight='balanced', n_estimators=100, max_depth=8, random_state=rnd_st), test = 1)

 | f1 = 0.61315 | roc_auc = 0.76447


Достигли желаемого результата - модель предсказывает достаточно хорошо (f1 = 0.62177).

В итоге мы добились неплохих результатов: 
- исследовали баланс классов,
- учли их дисбаланс,
- получили подтвержение качества модели, доказав это расчетом нужных метрик.